In [ ]:
library(Signac)
library(Seurat)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v79)
library(ensembldb)
suppressMessages(library(dplyr))
library('Matrix')
set.seed(1)

library(RColorBrewer)
library(ggplot2)
library(pheatmap)
library(cowplot)
library(reshape2)

In [ ]:
#### set enviroment
home = '~/farm/endometrium/data/scATAC/'
setwd(home)
outdir = paste0(home, 'seurat.output-S2D19/')
system(paste0('mkdir -p ', outdir, '/clusters/DARs'))
system(paste0('mkdir -p ', outdir, '/clusters/DARs_invivo'))
#### 


In [ ]:
### Load data
so = readRDS(file = paste0(outdir, '/data/M3_scRNAseq_labelsTransfer.rds'))

# Find differentially accessible peaks between clusters

Utilize logistic regression for DA, as suggested by Ntranos et al. 2018 for scRNA-seq data, 
and add the total number of fragments as a latent variable to mitigate the effect of differential sequencing depth on the result. 
We can also visualize these chromatin ‘markers’ on a violin plot, feature plot, dot plot, heat map, or any visualization tool in Seurat.



In [ ]:
# Generate GRanges object
GR = genes(EnsDb.Hsapiens.v79)

#switch back to working with peaks instead of gene activities
DefaultAssay(so) <- 'peaks'

for( cl in rev(levels(Idents(so))) ){
    message('Testing ', cl,'\n')
    da_peaks = FindMarkers(so, ident.1 = cl, test.use = "LR", 
                           min.pct = 0.1, 
                           latent.vars = 'nCount_peaks', 
                           logfc.threshold = 0.1)
    write.csv(da_peaks, file = paste0(outdir, '/clusters/DARs/', cl, '_LR.csv'))
    hist(da_peaks$avg_logFC, breaks = 100, main = paste0('cluster ', cl) )
    
    # Get closest genes
#     da_peaks = read.csv(paste0(outdir, '/clusters/DARs/', cl, '_LR.csv'), stringsAsFactors = F)
#     open_cl <- da_peaks$X
    open_cl <- rownames(da_peaks)
    open_cl = gsub('chr', '', open_cl)
    
    if( length(open_cl) == 0 )
        next()
    closest_genes_cl <- ClosestFeature(
        regions = open_cl,
        annotation = GR,
        sep = c(':', '-'))
    
    # add DARs info
    idx = match(closest_genes_cl$query_region, open_cl)
    closest_genes_cl$Query = open_cl[ idx ]
    closest_genes_cl$p_val = da_peaks$p_val[ idx ]
    closest_genes_cl$avg_logFC = da_peaks$avg_logFC[ idx ]
    closest_genes_cl$pct.1 = da_peaks$pct.1[ idx ]
    closest_genes_cl$pct.2 = da_peaks$pct.2[ idx ]
    closest_genes_cl$p_val_adj = da_peaks$p_val_adj[ idx ]
    
    closest_genes_cl = subset(closest_genes_cl, Query == query_region)[, -c(4,6) ]
    write.csv(closest_genes_cl, file = paste0(outdir, '/clusters/DARs/', cl, '_closest_genes.csv'), row.names =F)
}

In [ ]:
so = subset(so, cells = names(Idents(so))[ Idents(so) %in% c('ciliated', 'new_secretory', 'inter_PGRpos') ]  )

for( cl in c('new_secretory', 'inter_PGRpos', 'ciliated') ){
    message('Testing ', cl,'\n')
    da_peaks = FindMarkers(so, ident.1 = cl, test.use = "LR", 
                           min.pct = 0.1, 
                           latent.vars = 'nCount_peaks', 
                           logfc.threshold = 0.1)
    write.csv(da_peaks, file = paste0(outdir, '/clusters/DARs_invivo/', cl, '_LR.csv'))
    hist(da_peaks$avg_logFC, breaks = 100, main = paste0('cluster ', cl) )
    
    # Get closest genes
#     da_peaks = read.csv(paste0(outdir, '/clusters/DARs/', cl, '_LR.csv'), stringsAsFactors = F)
#     open_cl <- da_peaks$X
    open_cl <- rownames(da_peaks)
    open_cl = gsub('chr', '', open_cl)
    
    if( length(open_cl) == 0 )
        next()
    closest_genes_cl <- ClosestFeature(
        regions = open_cl,
        annotation = GR,
        sep = c(':', '-'))
    
    # add DARs info
    idx = match(closest_genes_cl$query_region, open_cl)
    closest_genes_cl$Query = open_cl[ idx ]
    closest_genes_cl$p_val = da_peaks$p_val[ idx ]
    closest_genes_cl$avg_logFC = da_peaks$avg_logFC[ idx ]
    closest_genes_cl$pct.1 = da_peaks$pct.1[ idx ]
    closest_genes_cl$pct.2 = da_peaks$pct.2[ idx ]
    closest_genes_cl$p_val_adj = da_peaks$p_val_adj[ idx ]
    
    closest_genes_cl = subset(closest_genes_cl, Query == query_region)[, -c(4,6) ]
    write.csv(closest_genes_cl, file = paste0(outdir, '/clusters/DARs_invivo/', cl, '_closest_genes.csv'), row.names =F)
}

# Enrichments

In [ ]:
# create_contingency_table = function(cl, GenSet){
#   cl_GenSet = intersect(GenSet, cl) %>% length(.)
#   nocl_GenSet =  unique(GenSet) %>% length(.)
#   noGS = setdiff(allGS, GenSet)
#   cl_outGenSet = intersect(noGS, cl) %>% length(.)
#   nocl_outGenSet = unique(noGS) %>% length(.)
#   m = matrix(c(cl_GenSet, nocl_GenSet,  cl_outGenSet,  nocl_outGenSet), 
#              ncol = 2, dimnames = list(c('Genset', 'All'), c('in_cl', 'total')) )
#   return(m)  
# }

create_contingency_table = function(cl, GenSet){
  cl_GenSet = intersect(GenSet, cl) %>% length(.)
  Allcl_GenSet =  intersect(GenSet, GENES) %>% length(.)
  noGenSet = setdiff(allGS, GenSet)
  cl_AllGenSet = intersect(noGenSet, cl) %>% length(.)
  Allcl_AllGenSet = intersect(noGenSet, GENES) %>% length(.)
  m = matrix(c(cl_GenSet,  cl_AllGenSet,  Allcl_GenSet, Allcl_AllGenSet), byrow = T,
             ncol = 2, dimnames = list(c('cl', 'Allcl'), c('GenSet', 'noGenSet')) )
  return(m)  
}


enrichment = function(cl, GenSet){
  funtable = create_contingency_table(cl, GenSet)
  out = NULL
  if ( funtable[1,1] > 1 ){
    ft = fisher.test(funtable)
    pvalue = ft$p.value
    estimate = ft$estimate
    conf.int = ft$conf.int
    out = data.frame(pvalue= pvalue, odds.ratio = estimate, min_confint = conf.int[1], max_confint = conf.int[2], n=funtable[1,1], stringsAsFactors = F)
  return(out)
  }
}


GENES = list()
for( cl in as.character(0:12) ){
    closest_genes_cl = read.csv(paste0(outdir, '/clusters/DARs/',cl,'_closest_genes.csv'), stringsAsFactors = F)
    closest_genes_cl = subset(closest_genes_cl, gene_name != "" )
    closest_genes_cl = subset(closest_genes_cl, distance < 5000)
    GENES[[cl]] = closest_genes_cl$gene_name
}
GENES = unlist(GENES) %>% unique(.)

## TFs

In [ ]:
viper_gset = get(load('~/farm/gsea/genesets/dorotheav2-top10scoring_VentoLab20200121.rdata'))
GS_pos = lapply(viper_gset, function(x) names(x$tfmode)[ as.numeric(x$tfmode) >= 0 ] )
GS_neg = lapply(viper_gset, function(x) names(x$tfmode)[ as.numeric(x$tfmode) < 0 ] )
GS_pos = lapply(GS_pos, intersect, GENES)
GS_neg = lapply(GS_neg, intersect, GENES)
GS_pos = GS_pos[ sapply(GS_pos, length) >= 3 ]
GS_neg = GS_neg[ sapply(GS_neg, length ) >= 3 ]
names(GS_pos) = paste0(names(GS_pos), '_pos')
names(GS_neg) = paste0(names(GS_neg), '_neg')
GS = append(GS_pos, GS_neg)
allGS = unlist(GS)

In [ ]:
# viper_gset = get(load('~/farm/gsea/genesets/dorotheav2-top10scoring_VentoLab20200121.rdata'))
# GS = lapply(viper_gset, function(x) names(x$tfmode) )
# GS = lapply(GS, intersect, GENES)
# GS = GS[ sapply(GS, length) >= 3 ]
# allGS = unlist(GS)

In [ ]:
GS$CSRNP1_E_pos

In [ ]:
results = list()

for( cl in as.character(0:12) ){
    message(cl)
    closest_genes_cl = read.csv(paste0(outdir, '/clusters/DARs/',cl,'_closest_genes.csv'), stringsAsFactors = F)
    closest_genes_cl = subset(closest_genes_cl, gene_name != "" )
    closest_genes_cl = subset(closest_genes_cl, distance < 5000)
    closest_genes_cl = closest_genes_cl[ order(closest_genes_cl$p_val_adj, decreasing = F), ]
    closest_genes_cl = closest_genes_cl[ ! duplicated(closest_genes_cl$gene_name),  ] 
    closest_genes_cl = subset(closest_genes_cl, p_val_adj < 0.05 & avg_logFC > 0.5)
    
    cl_genes = closest_genes_cl$gene_name
    ENRCH = lapply(GS, enrichment, cl=cl_genes) 
    print(cl_genes[1:5])
    fENRCH = ENRCH[ ! sapply(ENRCH, is.null) ] 
    df_ENRCH = melt(fENRCH, id.vars = colnames(fENRCH[[1]]) )
    names(df_ENRCH)[6] = 'TF'
    df_ENRCH$cl = cl
    if( nrow(df_ENRCH) > 0 )
        results[[cl]] = df_ENRCH
}
df = melt(results, id.vars = names(results[[1]]))
df = df[ order(df$pvalue) , -ncol(df) ]
write.csv(df, file = paste0(outdir, '/clusters/DARs/TFs_activities_GSEA.csv'), row.names = F, quote = F)

In [ ]:
results = list()

for( cl in c('new_secretory', 'ciliated', 'inter_PGRpos')  ){
    message(cl)
    closest_genes_cl = read.csv(paste0(outdir, '/clusters/DARs_invivo/',cl,'_closest_genes.csv'), stringsAsFactors = F)
    closest_genes_cl = subset(closest_genes_cl, gene_name != "" )
    closest_genes_cl = subset(closest_genes_cl, distance < 5000)
    closest_genes_cl = closest_genes_cl[ order(closest_genes_cl$p_val_adj, decreasing = F), ]
    closest_genes_cl = closest_genes_cl[ ! duplicated(closest_genes_cl$gene_name),  ] 
    closest_genes_cl = subset(closest_genes_cl, p_val_adj < 0.01 & avg_logFC > 0)
    
    cl_genes = closest_genes_cl$gene_name
    ENRCH = lapply(GS, enrichment, cl=cl_genes) 
    fENRCH = ENRCH[ ! sapply(ENRCH, is.null) ] 
    print(fENRCH$CSRNP1_E)
    df_ENRCH = melt(fENRCH, id.vars = colnames(fENRCH[[1]]) )
    names(df_ENRCH)[6] = 'TF'
    df_ENRCH$cl = cl
    if( nrow(df_ENRCH) > 0 )
        results[[cl]] = df_ENRCH
}
df = melt(results, id.vars = names(results[[1]]))
df = df[ order(df$pvalue) , -ncol(df) ]
write.csv(df, file = paste0(outdir, '/clusters/DARs_invivo/TFs_activities_GSEA.csv'), row.names = F, quote = F)

## pathways

In [ ]:
# Load Genesets
viper_gset = get(load('~/farm/gsea/genesets/msigDB_viper.rdata'))
# names(viper_gset) = paste0('msigDB_', names(viper_gset))
viper_gset = viper_gset[ c(grep("KEGG", names(viper_gset)), grep("REACTOME", names(viper_gset))) ]
# viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/harmonizome_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/TFrole_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/GObpUniprot_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/cellUniprot_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/DevelopUniprot_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/tissueExpressionUniprot_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/UniprotKeywords_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/Seuratccgenes_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/bgee_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/ensembl_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/ovarianDevelopment_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/stevant2019_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/zhang2018_oocyte_GC_DEsignatures_viper.rdata')))
viper_gset = append(x = viper_gset,  get(load('~/farm/gsea/genesets/TestisDevelopment_viper.rdata')))
idx2remove = unlist(sapply(c('DISEASE', 'TUMOR', 'CANCER', 'CARCINOMA', 'OMA_', 'OMA ', 'RESISTANCE', 'NEOPLA', 'VIRAL', 'CARIES','BRAIN', 
                             'RAPAMYCIN', 'INFLUENZA', 'CARCINOGENESIS', 'ASTHMA', '_FLU', 'SERUM', 'DISORDER', 'PROBIOTIC', '_DAY',
                             'ONCO', 'E_COLI', 'INFECTION', 'THERAPY', 'LEUKEMIA', 'TROPHY', 'OID_', 'MOUSE', 'FUSION', 'CIGenSetLATIN', 'METASTASIS',
                             'INJURY', 'TRANGenSetLANT', 'BACTER', 'FUNG', 'HEART', 'FUNG', 'TRANGenSetLANT', '_DN$'), 
                           grep, x= toupper(names(viper_gset))))
viper_gset = viper_gset[ - idx2remove ]


GS = lapply(viper_gset, function(x) names(x$tfmode) )
GS = lapply(GS, intersect, GENES)
GS = GS[ sapply(GS, length) > 3 ]
allGS = unlist(GS) %>% unique(.)

In [ ]:
results = list()

for( cl in rev(levels(Idents(so))) ){
    message(cl)
    closest_genes_cl = read.csv(paste0(outdir, '/clusters/DARs/',cl,'_closest_genes.csv'), stringsAsFactors = F)
    closest_genes_cl = subset(closest_genes_cl, gene_name != "" )
    closest_genes_cl = subset(closest_genes_cl, distance < 5000)
    closest_genes_cl = closest_genes_cl[ order(closest_genes_cl$p_val_adj, decreasing = F), ]
    closest_genes_cl = closest_genes_cl[ ! duplicated(closest_genes_cl$gene_name),  ] 
    closest_genes_cl = subset(closest_genes_cl, p_val_adj < 0.05 & avg_logFC > 0)
    
    cl_genes = closest_genes_cl$gene_name
    ENRCH = lapply(GS, enrichment, cl=cl_genes) 
    fENRCH = ENRCH[ ! sapply(ENRCH, is.null) ] 
    df_ENRCH = melt(fENRCH, id.vars = colnames(fENRCH[[1]]) )
    names(df_ENRCH)[6] = 'pathway'
    df_ENRCH$cl = cl
    if( nrow(df_ENRCH) > 0 )
        results[[cl]] = df_ENRCH
}
df = melt(results, id.vars = names(results[[1]]))
df = df[ order(df$pvalue) , -ncol(df) ]
write.csv(df, file = paste0(outdir, '/clusters/DARs/pathways_GSEA.csv'), row.names = F, quote = F)

In [ ]:
results = list()

for( cl in rev(levels(Idents(so))) ){
    message(cl)
    closest_genes_cl = read.csv(paste0(outdir, '/clusters/DARs_invivo/',cl,'_closest_genes.csv'), stringsAsFactors = F)
    closest_genes_cl = subset(closest_genes_cl, gene_name != "" )
    closest_genes_cl = subset(closest_genes_cl, distance < 5000)
    closest_genes_cl = closest_genes_cl[ order(closest_genes_cl$p_val_adj, decreasing = F), ]
    closest_genes_cl = closest_genes_cl[ ! duplicated(closest_genes_cl$gene_name),  ] 
    closest_genes_cl = subset(closest_genes_cl, p_val_adj < 0.05 & avg_logFC > 0)
    
    cl_genes = closest_genes_cl$gene_name
    ENRCH = lapply(GS, enrichment, cl=cl_genes) 
    fENRCH = ENRCH[ ! sapply(ENRCH, is.null) ] 
    df_ENRCH = melt(fENRCH, id.vars = colnames(fENRCH[[1]]) )
    names(df_ENRCH)[6] = 'pathway'
    df_ENRCH$cl = cl
    if( nrow(df_ENRCH) > 0 )
        results[[cl]] = df_ENRCH
}
df = melt(results, id.vars = names(results[[1]]))
df = df[ order(df$pvalue) , -ncol(df) ]
write.csv(df, file = paste0(outdir, '/clusters/DARs_invivo/pathways_GSEA.csv'), row.names = F, quote = F)